## Bhavesh Bhatt
[My YouTube channel](https://www.youtube.com/BhaveshBhatt8791)

# Installs

In [ ]:
!pip install -q gradio

# Imports

In [ ]:
import json
import requests
import pandas as pd
import gradio as gr
import librosa
import soundfile as sf
import torch
import warnings

# HuggingFace API Key

In [ ]:
with open('HF_SECRET_KEY.json') as f:
    hf_data = json.load(f)

In [ ]:
API_TOKEN = hf_data["API_KEY"]

# Defining End Points

In [ ]:
# https://api-inference.huggingface.co/docs/python/html/detailed_parameters.html

In [ ]:
ASR_API_URL = "https://api-inference.huggingface.co/models/facebook/wav2vec2-base-960h"
Q_A_API_URL = "https://api-inference.huggingface.co/models/google/tapas-base-finetuned-wtq"

In [ ]:
headers = {"Authorization": f"Bearer {API_TOKEN}"}

# Query Functions

In [ ]:
def q_n_a_query(payload):
    
    data = json.dumps(payload)
    response = requests.request("POST", 
                                Q_A_API_URL, 
                                headers=headers, 
                                data=data)
    return json.loads(response.content.decode("utf-8"))

In [ ]:
def asr_query(audio_file):
    
    sr, data = audio_file
    sf.write("temp.wav", data, 44100)
    with open("temp.wav", "rb") as f:
        data = f.read()
    response = requests.request("POST", 
                                ASR_API_URL, 
                                headers=headers, 
                                data=data)
    
    output = json.loads(response.content.decode("utf-8"))
    
    table_query = output["text"]
    
    data = q_n_a_query({
            "inputs": {
            "query": table_query,
            "table": table.to_dict('list')}})
    
    result = data["answer"]
    return table, table_query, result

# Load Data

In [ ]:
table = pd.read_csv("data.csv")

In [ ]:
table.head(3)

In [ ]:
table = table.astype(str)

# Define the Interface

In [ ]:
iface = gr.Interface(asr_query, "microphone", ["dataframe", 
                                               "text", 
                                               "text"])
iface.launch(debug=True)